# Query Expansion - Verbesserte Retrieval-Abdeckung durch Multiple Queries

## Was ist Query Expansion?

Query Expansion ist eine **Technik zur Verbesserung des Retrievals** in RAG-Systemen:

- **Problem**: Eine einzelne User-Query kann wichtige relevante Dokumente verpassen
- **Lösung**: Generiere mehrere semantisch ähnliche Query-Varianten und suche mit allen gleichzeitig
- **Resultat**: Breitere Abdeckung relevanter Dokumente durch verschiedene Formulierungen

### Warum ist Query Expansion wichtig?

**Problem mit Single-Query Retrieval:**
- Nutzer formulieren Fragen oft **unvollständig oder unpräzise**
- Dokumente verwenden möglicherweise **andere Begriffe oder Synonyme**
- Eine einzelne Query erfasst nicht alle **semantischen Perspektiven** einer Frage

**Beispiel (aus dem Haystack Tutorial):**
- Original Query: "green energy sources"
- Expanded Queries:
  - "renewable energy sources"
  - "sustainable power generation"
  - "eco-friendly energy options"
  - "clean energy resources"

Jede dieser Varianten kann **andere relevante Dokumente** finden, die die Original-Query verpasst hätte!

### Lösung: MultiQueryRetriever

Der **LangChain MultiQueryRetriever** automatisiert Query Expansion:
- Nutzt ein LLM, um automatisch mehrere Query-Varianten zu generieren
- Führt Retrieval für jede Variante durch
- Dedupliziert und kombiniert die Ergebnisse
- Liefert eine breitere Auswahl relevanter Dokumente

**Trade-offs:**
- ✅ Höhere **Recall** (findet mehr relevante Dokumente)
- ✅ Robuster gegenüber ungenauen Nutzer-Queries
- ✅ Erfasst verschiedene semantische Perspektiven
- ❌ **Mehr API-Calls** (ein LLM-Call für Query-Generierung + mehrere Retrievals)
- ❌ **Höhere Latenz** durch zusätzliche Schritte
- ❌ Kann auch **irrelevante Dokumente** einschließen

Daher ist Query Expansion besonders nützlich bei **komplexen oder mehrdeutigen Queries**!

## Setup: Wikipedia Energie-Artikel laden

Wir verwenden die gleichen Wikipedia-Artikel wie im [Haystack Query Expansion Tutorial](https://haystack.deepset.ai/cookbook/query-expansion):
- **Renewable Energy**: Solar, Wind, Hydroelectricity, etc.
- **Fossil Fuels**: Coal, Natural Gas, Oil
- **Related Topics**: Electric Vehicles, Batteries, Greenhouse Gases

Diese Artikel sind ideal, um Query Expansion zu demonstrieren, weil:
- Viele **Synonyme** existieren ("green energy", "renewable energy", "sustainable power")
- **Verschiedene Perspektiven** sind relevant (Technologie, Umwelt, Wirtschaft)
- Queries können **unpräzise** sein ("clean energy" könnte Solar, Wind, oder Hydro meinen)

In [ ]:
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb

print("📥 Lade Wikipedia-Artikel...")
print("   Installiere wikipedia package falls nötig: pip install wikipedia")

try:
    import wikipedia
except ImportError:
    print("⚠️  'wikipedia' package nicht gefunden!")
    print("   Installiere es mit: pip install wikipedia")
    raise

# Liste der Wikipedia-Artikel (wie im Haystack Tutorial)
article_titles = [
    "Electric_vehicle",
    "Electric_battery",
    "Solar_panel",
    "Wind_power",
    "Hydroelectricity",
    "Nuclear_power",
    "Coal",
    "Natural_gas",
    "Fossil_fuel",
    "Renewable_energy",
    "Greenhouse_gas",
    "Dam",
    "Tree"
]

print(f"   Lade {len(article_titles)} Artikel von Wikipedia...\n")

# Lade Wikipedia-Inhalte
raw_documents = []
for title in article_titles:
    try:
        page = wikipedia.page(title, auto_suggest=False)
        doc = Document(
            page_content=page.content,
            metadata={
                "source": "wikipedia",
                "title": title,
                "url": page.url
            }
        )
        raw_documents.append(doc)
        print(f"   ✓ {title}")
    except Exception as e:
        print(f"   ✗ {title} - Fehler: {e}")

print(f"\n✓ {len(raw_documents)} Artikel erfolgreich geladen")

In [ ]:
# Chunke die Dokumente in kleinere Abschnitte
print("✂️  Chunke Dokumente...")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)

documents = text_splitter.split_documents(raw_documents)

print(f"   ✓ {len(documents)} Chunks erstellt aus {len(raw_documents)} Artikeln")
print(f"   Durchschnittlich {len(documents) // len(raw_documents)} Chunks pro Artikel")

# Erstelle Vector Store mit Embeddings
print("\n🔢 Erstelle Embeddings und Vector Store...")

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Erstelle einen neuen ephemeral client
client = chromadb.EphemeralClient()

# Erstelle Chroma Vector Store
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="query_expansion_demo",
    client=client
)

print("✓ Vector Store erstellt")
print(f"  Anzahl Dokumente im Store: {len(documents)}")
print(f"\n📄 Beispiel-Dokument:")
print(f"  Titel: {documents[0].metadata['title']}")
print(f"  Content: {documents[0].page_content[:200]}...")

## Teil 1: Retrieval OHNE Query Expansion

Zuerst schauen wir uns an, wie ein normaler Single-Query Retrieval funktioniert und wo seine Limitationen liegen.

In [ ]:
# Query: Eine kurze, unpräzise Frage (wie im Haystack Tutorial)
query = "green energy sources"

# Retrieval mit Similarity Search (k=5 Dokumente)
results_without_expansion = vector_store.similarity_search_with_score(query, k=5)

print(f"Query: '{query}'")
print("\n" + "="*80)
print("ERGEBNISSE OHNE QUERY EXPANSION (Single Query)")
print("="*80 + "\n")

for rank, (doc, score) in enumerate(results_without_expansion, 1):
    print(f"Rang {rank} | Similarity: {score:.4f} | Artikel: {doc.metadata['title']}")
    print(f"Content: {doc.page_content[:150]}...")
    print("-" * 80)

### Beobachtung: Was ist das Problem?

Schaue dir die Ergebnisse genau an:

**Query:** "green energy sources"

Die Frage ist **unpräzise und kurz**. Was meint der Nutzer genau?
- Solar Energy?
- Wind Power?
- Hydroelectricity?
- Alle erneuerbaren Energien?

**Probleme mit Single-Query Retrieval:**
1. Die Query ist zu **unspezifisch** - "green energy" kann viele verschiedene Technologien bedeuten
2. **Synonyme** werden nicht erfasst ("renewable", "sustainable", "clean", "eco-friendly")
3. Verschiedene **semantische Perspektiven** werden nicht abgedeckt (Technologie vs. Umwelt vs. Wirtschaft)
4. Möglicherweise werden wichtige relevante Dokumente **übersehen**, weil sie andere Begriffe verwenden

**Beispiele für verpasste Synonyme:**
- "green energy" ≠ "renewable energy" (aber semantisch ähnlich)
- "sources" ≠ "power generation", "resources", "technologies"
- Spezifische Technologien wie "solar", "wind", "hydro" werden nicht direkt erwähnt

→ **Query Expansion kann helfen**, indem automatisch Varianten mit diesen Synonymen generiert werden!

## Teil 2: Retrieval MIT Query Expansion (MultiQueryRetriever)

Jetzt verwenden wir den LangChain MultiQueryRetriever, um automatisch mehrere Query-Varianten zu generieren.

In [ ]:
import logging
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

# Enable logging to see generated queries
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Erstelle LLM für Query-Generierung
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# Erstelle MultiQueryRetriever
# Dieser nutzt das LLM, um automatisch mehrere Query-Varianten zu generieren
base_retriever = vector_store.as_retriever(search_kwargs={"k": 5})
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=base_retriever,
    llm=llm
)

print("✓ MultiQueryRetriever erstellt")
print("  Das LLM wird automatisch mehrere Query-Varianten generieren")

In [ ]:
# Query Expansion in Aktion
# Der MultiQueryRetriever wird:
# 1. Mehrere Query-Varianten generieren (siehst du im Log)
# 2. Für jede Variante ein Retrieval durchführen
# 3. Alle Ergebnisse deduplizieren und kombinieren

query = "green energy sources"

print(f"Query: '{query}'")
print("\n" + "="*80)
print("ERGEBNISSE MIT QUERY EXPANSION (MultiQueryRetriever)")
print("="*80 + "\n")
print("⏳ Generiere Query-Varianten und führe Retrieval durch...\n")

# Invoke MultiQueryRetriever
# Achte auf die Log-Ausgabe oben - dort siehst du die generierten Queries!
results_with_expansion = multi_query_retriever.invoke(query)

print(f"\n✓ Gefunden: {len(results_with_expansion)} eindeutige Dokumente\n")
print("="*80 + "\n")

for rank, doc in enumerate(results_with_expansion, 1):
    print(f"Rang {rank} | Artikel: {doc.metadata['title']}")
    print(f"Content: {doc.page_content[:150]}...")
    print("-" * 80)

### Beobachtung: Was hat sich geändert?

Vergleiche die Ergebnisse vorher und nachher:

**Achte auf:**
1. **Anzahl der Dokumente**: Query Expansion findet oft mehr relevante Dokumente
2. **Diversität**: Die Dokumente decken verschiedene Energie-Technologien ab (Solar, Wind, Hydro, etc.)
3. **Generierte Queries** (siehe Log oben): Das LLM hat automatisch verschiedene Formulierungen erstellt

**Erwartetes Verhalten:**
- Mehr **relevante Dokumente** über erneuerbare Energien werden gefunden
- **Verschiedene Technologien** werden abgedeckt (Solar, Wind, Hydroelectricity, Nuclear)
- Das LLM hat Query-Varianten mit **Synonymen und Umformulierungen** erstellt
- **Höherer Recall**: Weniger Chance, wichtige Dokumente zu übersehen

**Mögliche generierte Queries (Beispiele aus dem Haystack Tutorial):**
- "renewable energy sources"
- "sustainable power generation"
- "eco-friendly energy options"
- "clean energy resources"
- "alternative energy technologies"

**Warum ist das besser?**
- **"renewable"** findet den "Renewable_energy" Artikel, der bei "green" vielleicht nicht top-ranked war
- **"sustainable"** und **"clean"** erweitern das semantische Feld
- **"power generation"** statt "sources" findet Dokumente mit anderen Formulierungen
- Jede Query-Variante findet potenziell andere relevante Dokumente!

## Teil 3: Vergleich und Analyse

Schauen wir uns die Unterschiede systematisch an.

In [ ]:
import pandas as pd

# Dokumente aus beiden Methoden extrahieren
single_query_doc_ids = {(doc.metadata['title'], doc.page_content[:100]) for doc, _ in results_without_expansion}
multi_query_doc_ids = {(doc.metadata['title'], doc.page_content[:100]) for doc in results_with_expansion}

# Analyse
only_in_single = single_query_doc_ids - multi_query_doc_ids
only_in_multi = multi_query_doc_ids - single_query_doc_ids
in_both = single_query_doc_ids & multi_query_doc_ids

print("="*80)
print("VERGLEICH: Single Query vs. Query Expansion")
print("="*80 + "\n")

print(f"Dokumente OHNE Expansion (Single Query): {len(results_without_expansion)}")
print(f"Dokumente MIT Expansion (Multi Query):   {len(results_with_expansion)}")
print(f"\nÜberlappung (in beiden):                 {len(in_both)}")
print(f"Nur in Single Query:                      {len(only_in_single)}")
print(f"Nur in Multi Query:                       {len(only_in_multi)}")

if only_in_multi:
    print("\n" + "-"*80)
    print("NEU GEFUNDENE DOKUMENTE durch Query Expansion:")
    print("-"*80 + "\n")
    
    # Zeige neu gefundene Dokumente
    multi_dict = {(doc.metadata['title'], doc.page_content[:100]): doc for doc in results_with_expansion}
    
    for key in only_in_multi:
        doc = multi_dict[key]
        print(f"📄 Artikel: {doc.metadata['title']}")
        print(f"   Content: {doc.page_content[:200]}...")
        print("-" * 80)
        
print(f"\n💡 Query Expansion hat {len(only_in_multi)} neue relevante Dokumente gefunden!")
print(f"   Das sind {len(only_in_multi) / len(results_without_expansion) * 100:.0f}% mehr Dokumente.")

## Teil 4: Custom Prompt für Query Expansion

Der MultiQueryRetriever verwendet standardmäßig ein generisches Prompt. Du kannst aber ein **eigenes Prompt** definieren, um die Query-Generierung anzupassen.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import BaseOutputParser
from typing import List

# Custom Output Parser: Konvertiert LLM-Output in eine Liste von Queries
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a list of lines."""
    
    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return [line.strip() for line in lines if line.strip()]

# Custom Prompt: Spezialisiert auf Energie-Themen
CUSTOM_QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Du bist ein Experte für Energie-Technologien und erneuerbare Energien.
Deine Aufgabe ist es, alternative Formulierungen einer Nutzer-Frage zu generieren.

Generiere 4 alternative Versionen der folgenden Frage.
Die Fragen sollten verschiedene Perspektiven und Synonyme nutzen.
Achte auf den Kontext von erneuerbaren Energien, Nachhaltigkeit, und Energie-Technologien.

Berücksichtige Synonyme wie:
- "green" → "renewable", "sustainable", "clean", "eco-friendly"
- "sources" → "technologies", "power generation", "resources", "systems"
- Spezifische Technologien: Solar, Wind, Hydro, Nuclear, etc.

Original-Frage: {question}

Alternative Fragen (eine pro Zeile):""",
)

# Erstelle Custom LLM Chain
output_parser = LineListOutputParser()
llm_chain = CUSTOM_QUERY_PROMPT | llm | output_parser

# Erstelle MultiQueryRetriever mit Custom Chain
custom_multi_query_retriever = MultiQueryRetriever(
    retriever=base_retriever,
    llm_chain=llm_chain,
    parser_key="lines"
)

print("✓ Custom MultiQueryRetriever erstellt")
print("  Verwendet ein spezialisiertes Prompt für Energie-Themen")

In [ ]:
# Teste den Custom Retriever
query = "green energy sources"

print(f"Query: '{query}'")
print("\n" + "="*80)
print("ERGEBNISSE MIT CUSTOM QUERY EXPANSION")
print("="*80 + "\n")
print("⏳ Generiere Query-Varianten mit Custom Prompt...\n")

results_custom = custom_multi_query_retriever.invoke(query)

print(f"\n✓ Gefunden: {len(results_custom)} eindeutige Dokumente\n")
print("="*80 + "\n")

for rank, doc in enumerate(results_custom, 1):
    print(f"Rang {rank} | Artikel: {doc.metadata['title']}")
    print(f"Content: {doc.page_content[:150]}...")
    print("-" * 80)

### Beobachtung: Custom Prompt

Vergleiche die generierten Queries (siehe Log) mit dem Standard-Prompt:

**Custom Prompt Vorteile:**
- ✅ **Domain-spezifisch**: Fokus auf Energie-Technologien und erneuerbare Energien
- ✅ **Explizite Synonyme**: Prompt gibt konkrete Synonym-Beispiele ("green" → "renewable", "sustainable")
- ✅ **Kontrollierbar**: Du bestimmst den Stil und die Anzahl der Queries
- ✅ **Bessere Relevanz**: Generierte Queries passen besser zur Energie-Domäne

**Vergleich der generierten Queries:**

**Standard Prompt:**
- Eher generisch und könnte beliebige Themen abdecken
- Keine Domain-spezifischen Hinweise

**Custom Prompt:**
- Explizit auf Energie-Themen fokussiert
- Gibt Synonym-Beispiele vor ("renewable", "sustainable", "clean")
- Erwähnt spezifische Technologien (Solar, Wind, Hydro)
- Führt zu präziseren und relevanteren Query-Varianten

**Wann Custom Prompt verwenden?**
- Spezielle **Fachdomäne** (z.B. Medizin, Recht, Energie, Technik)
- Wenn du **bekannte Synonyme** in der Domäne hast
- Wenn Standard-Queries zu **generisch** sind
- Kontrolle über **Anzahl und Stil** der generierten Queries

## Teil 5: Verschiedene Queries testen

Probiere Query Expansion mit verschiedenen Arten von Fragen aus.

In [ ]:
def test_query_expansion(query_text: str, retriever=multi_query_retriever):
    """Helper function to test a query with expansion"""
    print(f"\n{'='*80}")
    print(f"Query: '{query_text}'")
    print("="*80 + "\n")
    
    # Single query
    single_results = vector_store.similarity_search(query_text, k=5)
    print(f"OHNE Expansion: {len(single_results)} Dokumente")
    for doc in single_results[:3]:  # Zeige nur top 3
        print(f"  - {doc.metadata['title']}")
    if len(single_results) > 3:
        print(f"  ... und {len(single_results) - 3} weitere")
    
    # Multi query
    print("\n⏳ MIT Expansion...\n")
    multi_results = retriever.invoke(query_text)
    print(f"\nMIT Expansion: {len(multi_results)} Dokumente")
    
    # Gruppiere nach Artikel-Titel
    articles = {}
    for doc in multi_results:
        title = doc.metadata['title']
        if title not in articles:
            articles[title] = []
        articles[title].append(doc)
    
    for title, docs in list(articles.items())[:5]:  # Zeige top 5 Artikel
        print(f"  - {title} ({len(docs)} chunks)")
    if len(articles) > 5:
        print(f"  ... und {len(articles) - 5} weitere Artikel")
    
    # Comparison
    improvement = len(multi_results) - len(single_results)
    print(f"\n📊 Unterschied: {improvement:+d} Dokumente ({improvement / len(single_results) * 100:.0f}% mehr)")
    print("-"*80)

# Test verschiedene Queries
test_queries = [
    "climate change impact",
    "solar power technology",
    "fossil fuel alternatives",
]

for q in test_queries:
    test_query_expansion(q)

## Integration in dein RAG-System

Du hast bereits eine Custom Query Expansion Implementierung in [query_expansion_retriever.py](../src/advanced_rag/backend/nodes/query_expansion_retriever.py).

### Unterschied: Custom Implementation vs. MultiQueryRetriever

**Deine Custom Implementation:**
```python
class QueryExpansionRetriever(BaseNode):
    - Nutzt structured output (Pydantic) für Query-Generierung
    - Integriert in dein State-Management
    - Custom Prompt aus query_expansion_prompt.py
    - Deduplizierung mit _unique_documents()
```

**LangChain MultiQueryRetriever:**
```python
MultiQueryRetriever.from_llm()
    - Standard LangChain Komponente
    - Einfacher zu verwenden
    - Automatische Deduplizierung
    - Logging integriert
```

### Empfehlung:

Behalte deine Custom Implementation, weil:
- ✅ Sie bereits in dein System integriert ist
- ✅ Du volle Kontrolle über das Prompt hast
- ✅ Structured Output ist robuster als String Parsing
- ✅ Sie mit deinem State-Management funktioniert

Du kannst aber Ideen aus dem MultiQueryRetriever übernehmen:
- Logging der generierten Queries
- Alternative Deduplizierungs-Strategien
- Fehlerbehandlung

## Übung: Query Expansion aktivieren und evaluieren

### Schritt 1: Query Expansion im RAG-System aktivieren

Öffne [query_routing_graph.py](../src/advanced_rag/backend/graphs/query_routing_graph.py) und aktiviere den QueryExpansionRetriever als Retriever-Option.

### Schritt 2: Backend starten

```bash
uv run --env-file .env python -m src.advanced_rag.backend.main
```

### Schritt 3: Im Frontend testen

Teste mit kurzen, unpräzisen Queries wie:
- "andrena methoden"
- "team entwicklung"
- "agile praktiken"

### Schritt 4: Evaluation durchführen

Führe eine systematische Evaluation durch:

**OHNE Query Expansion:**
```bash
# Deaktiviere Query Expansion im Graph
uv run --env-file .env python src/advanced_rag/evaluation/evaluate_dataset.py
```

**MIT Query Expansion:**
```bash
# Aktiviere Query Expansion im Graph
uv run --env-file .env python src/advanced_rag/evaluation/evaluate_dataset.py
```

### Schritt 5: Metriken vergleichen

Schaue in Langfuse und vergleiche:

| Metrik              | OHNE Query Expansion | MIT Query Expansion | Verbesserung |
|---------------------|----------------------|---------------------|---------------|
| Context Precision   |          ?           |          ?          |      ?%       |
| Context Recall      |          ?           |          ?          |      ?%       |
| Answer Relevancy    |          ?           |          ?          |      ?%       |
| Faithfulness        |          ?           |          ?          |      ?%       |

### Erwartete Ergebnisse:

**Context Recall sollte steigen:**
- Query Expansion findet mehr relevante Dokumente durch verschiedene Formulierungen

**Context Precision könnte leicht sinken:**
- Mehr Dokumente bedeuten auch mehr Chance für irrelevante Treffer

**Answer Relevancy sollte steigen:**
- Breiterer Context ermöglicht umfassendere Antworten

**Trade-off beachten:**
- Höhere Kosten (mehr LLM-Calls)
- Höhere Latenz (Query-Generierung + mehrere Retrievals)
- Aber bessere Abdeckung bei komplexen Fragen

## Kombination: Query Expansion + Reranking

Die **optimale Pipeline** kombiniert beide Techniken:

```
User Query
    ↓
Query Expansion (generiere 4-5 Varianten)
    ↓
Initial Retrieval (k=10 pro Query = 40-50 Kandidaten)
    ↓
Deduplizierung (einzigartige Dokumente)
    ↓
Reranking mit CrossEncoder (sortiere nach Relevanz)
    ↓
Top-K Selection (z.B. beste 5 Dokumente)
    ↓
LLM Generation
```

### Vorteile der Kombination:

**Query Expansion:**
- ✅ Höherer **Recall** (findet mehr relevante Dokumente)
- ✅ Robuster gegenüber **unpräzisen Queries**

**Reranking:**
- ✅ Höhere **Precision** (sortiert irrelevante Dokumente aus)
- ✅ Bessere **semantische Relevanz**

**Zusammen:**
- ✅ **Hoher Recall UND hohe Precision**
- ✅ Robust gegen verschiedene Query-Formulierungen
- ✅ Beste Dokumente landen im Context
- ❌ Höhere Kosten und Latenz

### Implementation:

Erweitere deinen QueryExpansionRetriever:

```python
class QueryExpansionWithRerankingRetriever(BaseNode):
    def __init__(self, llm, vector_store, cross_encoder):
        self.llm = llm
        self.vector_store = vector_store
        self.cross_encoder = cross_encoder
    
    def traverse(self, state: State) -> State:
        original_query = state.get_current_question()
        
        # 1. Expand queries
        expanded_queries = self._expand_queries(original_query)
        
        # 2. Retrieve with all queries
        documents = self.retrieve_documents_from(expanded_queries)
        
        # 3. Rerank with CrossEncoder
        reranked_documents = self._rerank(original_query, documents)
        
        # 4. Keep top-k
        state.context = reranked_documents[:CONTEXT_SIZE_AFTER_RERANKING]
        return state
```

## Zusammenfassung

### Was hast du gelernt?

1. **Query Expansion verbessert Recall:**
   - Generiert automatisch mehrere Formulierungen einer Query
   - Findet Dokumente mit verschiedenen Begriffen und Synonymen
   - Robuster gegenüber unpräzisen Nutzer-Fragen

2. **LangChain MultiQueryRetriever:**
   - Einfache Integration mit `.from_llm()`
   - Automatische Deduplizierung
   - Anpassbar mit Custom Prompts

3. **Custom Prompts wichtig für:**
   - Domain-spezifische Anwendungen
   - Nicht-englische Sprachen
   - Kontrolle über Stil und Anzahl der Queries

4. **Trade-offs beachten:**
   - Höhere Kosten (LLM-Call + mehrere Retrievals)
   - Höhere Latenz
   - Aber deutlich bessere Abdeckung

5. **Best Practice: Kombination mit Reranking**
   - Query Expansion für hohen Recall
   - Reranking für hohe Precision
   - Beste Ergebnisse bei komplexen Queries

### Nächste Schritte:

- ✅ Teste Query Expansion in deinem RAG-System
- ✅ Evaluiere mit echten Test-Queries
- ✅ Experimentiere mit Custom Prompts für deine Domäne
- ✅ Kombiniere mit Reranking für optimale Ergebnisse
- ✅ Monitore Kosten und Latenz in Produktion

Viel Erfolg beim Optimieren deines RAG-Systems! 🚀